## Pandas analysis

This exercise consists in analyzing a dataset containg timing information from a series of Time-to-Digital-Converters (TDC) implemented in a couple of FPGAs. Each measurement (i.e. each row of the input file) consists of a flag that specifies the type of message ('HEAD', which in this case is always 1), two addresses of the TDC providing the signal ('FPGA' and 'TDC_CHANNEL'), and the timing information ('ORBIT_CNT', 'BX_COUNTER', and 'TDC_MEAS'). Each TDC count corresponds to 25/30 ns, whereas a unit of BX_COUNTER corresponds to 25 ns, and the ORBIT_CNT is increased every 'x' BX_COUNTER. This allows to store the time in a similar way to hours, minutes and seconds.

In [1]:
# If you didn't download it yet, please get the relevant file now!
!wget https://www.dropbox.com/s/xvjzaxzz3ysphme/data_000637.txt -P ~/data/

/bin/sh: wget: command not found


In [2]:
import pandas as pd
import numpy as np

1\. Create a Pandas DataFrame reading N rows of the 'data_000637.txt' dataset. Choose N to be smaller than or equal to the maximum number of rows and larger that 10k.

2\. Find out the number of BX in a ORBIT (the value 'x').

3\. Find out how much the data taking lasted. You can either make an estimate based on the fraction of the measurements (rows) you read, or perform this check precisely by reading out the whole dataset.

4\. Create a new column with the absolute time in ns (as a combination of the other three columns with timing information).

5\. Replace the values (all 1) of the HEAD column randomly with 0 or 1.

6\. Create a new DataFrame that contains only the rows with HEAD=1.

7\. Make two occupancy plots (one for each FPGA), i.e. plot the number of counts per TDC channel

8\. Use the groupby method to find out the noisy channels, i.e. the TDC channels with most counts (say the top 3)

9\. Count the number of unique orbits. Count the number of unique orbits with at least one measurement from TDC_CHANNEL=139

In [3]:
#1)Create a Pandas DataFrame reading N rows of the 'data_000637.txt' dataset. 
#Choose N to be smaller than or equal to the maximum number of rows and larger that 10k.

#read the data and create a dataframe
N=50000
df=pd.read_csv("Data/data_000637.txt",nrows=N)

#check dataframe
print(df.index)
print(df.columns)
df

RangeIndex(start=0, stop=50000, step=1)
Index(['HEAD', 'FPGA', 'TDC_CHANNEL', 'ORBIT_CNT', 'BX_COUNTER', 'TDC_MEAS'], dtype='object')


,HEAD,FPGA,TDC_CHANNEL,ORBIT_CNT,BX_COUNTER,TDC_MEAS
0,1,0,123,3869200167,2374,26
1,1,0,124,3869200167,2374,27
2,1,0,63,3869200167,2553,28
3,1,0,64,3869200167,2558,19
4,1,0,64,3869200167,2760,25
...,...,...,...,...,...,...
49995,1,1,2,3869200664,1103,4
49996,1,1,4,3869200664,1105,14
49997,1,0,59,3869200664,1191,17
49998,1,0,32,3869200664,1191,1


In [4]:
#2)Find out the number of BX in a ORBIT (the value 'x').

#find the max value for BX_COUNTER
max_bx=df.loc[:,'BX_COUNTER'].max() 

x=max_bx + 1 #how many BX_COUNT are inside 1 ORBIT
print("x=",x)

#check in dataframe if orbit increases
indeces=df[df['BX_COUNTER'] == max_bx].index #select indeces corrispondig to BX_COUNTER==max_bx
for i in range(0,2*len(indeces+1),2):indeces=indeces.insert(i+1,indeces[i]+1) #add following indeces
df.loc[indeces,['ORBIT_CNT','BX_COUNTER']] #check if ORBIT_CNT is increased

x= 3564


,ORBIT_CNT,BX_COUNTER
2893,3869200195,3563
2894,3869200196,0
4073,3869200207,3563
4074,3869200208,8
4461,3869200211,3563
4462,3869200211,3555
4745,3869200214,3563
4746,3869200214,3562
4747,3869200214,3563
4748,3869200215,1


In [5]:
#3) Find out how much the data taking lasted. 
#You can either make an estimate based on the fraction of the measurements (rows) you read, 
#or perform this check precisely by reading out the whole dataset

#read the whole dataset and create a DataFrame
data = pd.read_csv("Data/data_000637.txt") 

#index of last element
last_element=data.index.stop - 1 
print("Last Element:\n",data.loc[last_element,['ORBIT_CNT','BX_COUNTER','TDC_MEAS']],"\n")#print last element timing

#define time like variables reading the elements from dataframe
hours=data.iloc[last_element,3]
minutes=data.iloc[last_element,4] 
seconds=data.iloc[last_element,5] 

#calculate total timing
total_timing=(25*seconds)/30 + 25*minutes + hours*x*25
print("Total timing:\n",total_timing,"ns\n",total_timing/1e9,"s\n",total_timing/(60e9),"min\n",total_timing/(12e11),"h\n",total_timing/(288e11),"days")

Last Element:
 ORBIT_CNT     3869211171
BX_COUNTER           762
TDC_MEAS              18
Name: 1310719, dtype: int64 

Total timing:
 344746715355165.0 ns
 344746.715355165 s
 5745.77858925275 min
 287.2889294626375 h
 11.970372060943228 days


In [6]:
#4)Create a new column with the absolute time in ns (as a combination of the other three columns with timing information).

#create brand new column
data['ABSOLUTE_TIME'] = (25*data['TDC_MEAS'])/30 + 25*data['BX_COUNTER'] + data['ORBIT_CNT']*x*25

#check the previous calculation
print(data.iloc[last_element,6]==total_timing)

data

True


,HEAD,FPGA,TDC_CHANNEL,ORBIT_CNT,BX_COUNTER,TDC_MEAS,ABSOLUTE_TIME
0,1,0,123,3869200167,2374,26,3.447457e+14
1,1,0,124,3869200167,2374,27,3.447457e+14
2,1,0,63,3869200167,2553,28,3.447457e+14
3,1,0,64,3869200167,2558,19,3.447457e+14
4,1,0,64,3869200167,2760,25,3.447457e+14
...,...,...,...,...,...,...,...
1310715,1,0,62,3869211171,762,14,3.447467e+14
1310716,1,1,4,3869211171,763,11,3.447467e+14
1310717,1,0,64,3869211171,764,0,3.447467e+14
1310718,1,0,139,3869211171,769,0,3.447467e+14


In [7]:
#5) Replace the values (all 1) of the HEAD column randomly with 0 or 1.

#working on the reduced dataframe, N=50000

#replace the values
df['HEAD']=pd.Series([np.random.choice([0,1]) for i in range(N)])
df

,HEAD,FPGA,TDC_CHANNEL,ORBIT_CNT,BX_COUNTER,TDC_MEAS
0,1,0,123,3869200167,2374,26
1,0,0,124,3869200167,2374,27
2,0,0,63,3869200167,2553,28
3,0,0,64,3869200167,2558,19
4,1,0,64,3869200167,2760,25
...,...,...,...,...,...,...
49995,1,1,2,3869200664,1103,4
49996,1,1,4,3869200664,1105,14
49997,0,0,59,3869200664,1191,17
49998,1,0,32,3869200664,1191,1


In [8]:
#6) Create a new DataFrame that contains only the rows with HEAD=1.

#creating dataframe and dropping NaN values
data_frame=pd.DataFrame(df[df['HEAD']==1],index=df.index).dropna(how='any') 

#check dimensions
print(df.shape,data_frame.shape)

data_frame

(50000, 6) (24882, 6)


,HEAD,FPGA,TDC_CHANNEL,ORBIT_CNT,BX_COUNTER,TDC_MEAS
0,1.0,0.0,123.0,3.869200e+09,2374.0,26.0
4,1.0,0.0,64.0,3.869200e+09,2760.0,25.0
5,1.0,0.0,63.0,3.869200e+09,2762.0,4.0
14,1.0,0.0,56.0,3.869200e+09,2789.0,3.0
15,1.0,1.0,139.0,3.869200e+09,2797.0,0.0
...,...,...,...,...,...,...
49994,1.0,1.0,52.0,3.869201e+09,1102.0,16.0
49995,1.0,1.0,2.0,3.869201e+09,1103.0,4.0
49996,1.0,1.0,4.0,3.869201e+09,1105.0,14.0
49998,1.0,0.0,32.0,3.869201e+09,1191.0,1.0


In [9]:
#8) Use the groupby method to find out the noisy channels, 
#i.e. the TDC channels with most counts (say the top 3)

#working on the reduced dataframe, N=50000

#reset all the heads to 1
df['HEAD']=1 

#sum the 'heads' values to count how many times each channel appears
groups=df.groupby('TDC_CHANNEL').sum() 

#top 3 noisy channels
noisy_channels=groups['HEAD'].nlargest(3) 
print(noisy_channels)

TDC_CHANNEL
139    4175
64     2523
63     2472
Name: HEAD, dtype: int64


In [10]:
#9)Count the number of unique orbits with at least one measurement from TDC_CHANNEL=139

#working on the reduced dataframe, N=50000

#select elements with TDC_CHANNEL==139
df=df[df['TDC_CHANNEL']==139] 

#number of unique orbits
orbits=df['ORBIT_CNT'].nunique() 

print("Number of unique orbits with at least one measurement from channel 139:",orbits)

Number of unique orbits with at least one measurement from channel 139: 498
